In [2]:
# coding: utf-8
from os import listdir, makedirs
from os.path import join, isfile, isdir, exists
import pandas as pd
import gc
from gensim.corpora import Dictionary, MmCorpus
from gensim.models import CoherenceModel, TfidfModel, LdaModel, LdaMulticore
from gensim.models.wrappers import LdaMallet
from gensim.models.hdpmodel import HdpModel, HdpTopicFormatter
from gensim.models.callbacks import CoherenceMetric, DiffMetric, PerplexityMetric, ConvergenceMetric
from itertools import chain, islice
from constants import (
    DATA_BASE, FULL_PATH, ETL_PATH, NLP_PATH, SMPL_PATH, POS, NOUN, PROPN, TOKEN, HASH, SENT_IDX, PUNCT
)
import logging
import json
import numpy as np
import visdom
vis = visdom.Visdom()

#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
#pd.options.display.max_rows = 2001

In [3]:
def docs_to_lists(token_series):
    return tuple(token_series.tolist())

def docs2corpora(documents, tfidf=False, stopwords=None, filter_below=5, filter_above=0.5,
                split=False, max_test_size_rel=0.1, max_test_size_abs=5000):

    print('building dictionary')
    dictionary = Dictionary(documents)
    dictionary.filter_extremes(no_below=filter_below, no_above=filter_above)

    # filter some noice (e.g. special characters)
    if stopwords:
        stopword_ids = [dictionary.token2id[token] for token in stopwords]
        dictionary.filter_tokens(bad_ids=stopword_ids, good_ids=None)
        
    length = len(documents)
    corpora = dict()
    if split:
        if length*max_test_size_rel < max_test_size_abs:
            split1 = int(length*(1-(2*max_test_size_rel)))
            split2 = int(length*(1-max_test_size_rel))
        else:
            split1 = length-(2*max_test_size_abs)
            split2 = length-max_test_size_abs
        training_texts = documents[:split1]
        holdout_texts = documents[split1:split2]
        test_texts = documents[split2:]
        print(
            f'split dataset. size of:',
            f'train_set={len(training_texts)},',
            f'val_set={len(holdout_texts)},',
            f'test_set={len(test_texts)},'
        )
        corpora['training_corpus'] = [dictionary.doc2bow(text) for text in training_texts]
        corpora['holdout_corpus'] = [dictionary.doc2bow(text) for text in holdout_texts]
        corpora['test_corpus'] = [dictionary.doc2bow(text) for text in test_texts]
    else:
        training_texts = documents
        corpora['training_corpus'] = [dictionary.doc2bow(text) for text in training_texts]
        corpora['holdout_corpus'], corpora['test_corpus'] = None, None

    if tfidf:
        for key, bow_corpus in corpora.items():
            tfidf_model = TfidfModel(bow_corpus)
            corpora[key] = tfidf_model[bow_corpus]
    return corpora, dictionary

In [4]:
datasets = {
    'E': 'Europarl',
    'FA': 'FAZ_combined',
    'FO': 'FOCUS_cleansed',
    'O': 'OnlineParticipation',
    'P': 'PoliticalSpeeches',
    'dewi': 'dewiki',
    'dewa': 'dewac',
}
goodids = {
    # filetered via some fixed rules and similarity measure to character distribution
    'dewac': join(ETL_PATH, 'dewac_good_ids.pickle'),
    'dewiki': join(ETL_PATH, 'dewiki_good_ids.pickle'),
    # the samples contain only a small subset of all articles
    # the reason for this is that the samples are roughly equal in size per category
    # 'FAZ_combined': join(ETL_PATH, 'FAZ_document_sample3.pickle'),
    # 'FOCUS_cleansed': join(ETL_PATH, 'FOCUS_document_sample3.pickle'),
}
bad_tokens = {
    'Europarl': [
        'E.', 'Kerr', 'The', 'la', 'ia', 'For', 'Ieke', 'the',
    ],
    'FAZ_combined': [
        'S.', 'j.reinecke@faz.de', 'B.',
    ],
    'FOCUS_cleansed': [],
    'OnlineParticipation': [
        'Re', '@#1', '@#2', '@#3', '@#4', '@#5', '@#6', '@#7', '@#8', '@#9', '@#1.1', 'Für', 'Muss',
        'etc', 'sorry', 'Ggf', 'u.a.', 'z.B.', 'B.', 'stimmt', ';-)', 'lieber', 'o.', 'Ja', 'Desweiteren',
    ],
    'PoliticalSpeeches': [],
    'dewiki': [],
    'dewac': [],
}
all_bad_tokens = set(chain(*bad_tokens.values()))

In [5]:
dataset = datasets['O']
print('dataset:', dataset)
print('-' * 5)

sub_dir = 'dewiki' if dataset.startswith('dewi') else 'wiki_phrases'
dir_path = join(SMPL_PATH, sub_dir)
files = sorted([f for f in listdir(dir_path) if f.startswith(dataset)])

keepids = None
if dataset in goodids:
    keepids = pd.read_pickle(goodids[dataset])

nb_words = 0
documents = []
for name in files:
    gc.collect()
    full_path = join(dir_path, name)
    if not isfile(full_path):
        continue

    print('reading', name)
    df = pd.read_pickle(join(dir_path, name))
    print('    initial number of words:', len(df))
    if keepids is not None:
        # some datasets have already been filtered so you may not see a difference in any case
        df = df[df.hash.isin(keepids.index)]

    # fixing bad POS tagging
    mask = df.token.isin(list('[]<>/–%'))
    df.loc[mask, POS] = PUNCT

    # using only certain POS tags
    reduction_pos_set = {NOUN, PROPN, 'NER', 'NPHRASE'}
    df = df[df.POS.isin(reduction_pos_set)]
    df[TOKEN] = df[TOKEN].map(lambda x: x.strip('-/'))
    df = df[df.token.str.len() > 1]
    df = df[~df.token.isin(all_bad_tokens)]
    nb_words += len(df)
    print('    remaining number of words:', len(df))
    # groupby sorts the documents by hash-id which is equal to shuffeling the dataset before building the model
    df = df.groupby([HASH])[TOKEN].agg(docs_to_lists)
    print('    number of documents:', len(df))
    documents += df.values.tolist()

nb_docs = len(documents)
print('-' * 5)
print('total number of documents:', nb_docs)
print('total number of words:', nb_words)
stats = dict(dataset=dataset, pos_set=sorted(reduction_pos_set), nb_docs=nb_docs, nb_words=nb_words)
del keepids, files
gc.collect();

dataset: OnlineParticipation
-----
reading OnlineParticipation_simple_wiki_phrases.pickle
    initial number of words: 1661338
    remaining number of words: 374987
    number of documents: 25663
-----
total number of documents: 25663
total number of words: 374987


In [8]:
# save full dataset as bow/tfidf corpus in Matrix Market format, including dictionary
use_tfidf = False
corpus_type = 'tfidf' if use_tfidf else 'bow'
split = False
split_type = 'trainset' if split else 'fullset'
corpora, dictionary = docs2corpora(
    documents, tfidf=use_tfidf,
    filter_below=5, filter_above=0.5,
    split=split,
)
corpus = corpora['training_corpus']
file_name = f'{dataset}_{split_type}_nouns_{corpus_type}'
corpus_path = join(ETL_PATH, 'LDAmodel', file_name + '.mm')
print(f'saving {corpus_path}')
MmCorpus.serialize(corpus_path, corpus)
dict_path = join(ETL_PATH, 'LDAmodel', file_name + '.dict')
print(f'saving {dict_path}')
dictionary.save(dict_path)
doc_path = join(ETL_PATH, 'LDAmodel', file_name.rstrip(f'{corpus_type}') + 'texts.json')
print(f'saving {doc_path}')
with open(doc_path, 'w') as fp:
    json.dump(documents, fp, ensure_ascii=False)

building dictionary
saving ../data/preprocessed/LDAmodel/OnlineParticipation_fullset_nouns_bow.mm
saving ../data/preprocessed/LDAmodel/OnlineParticipation_fullset_nouns_bow.dict
saving ../data/preprocessed/LDAmodel/OnlineParticipation_fullset_nouns_texts.json


In [5]:
# initialize training and test data
corpora, dictionary = docs2corpora(
    documents, tfidf=False,
    #stopwords=bad_tokens[dataset],  # stopword removale has been moved to the pandas preprocessing pipeline
    filter_below=5, filter_above=0.5,
    split=True,
)
training_corpus = corpora['training_corpus']
holdout_corpus = corpora['holdout_corpus']
test_corpus = corpora['test_corpus']

def init_callbacks(viz_env=None, title_suffix=''):
    # define perplexity callback for hold_out and test corpus
    pl_holdout = PerplexityMetric(
        corpus=holdout_corpus, logger="visdom", viz_env=viz_env, title="Perplexity (hold_out)"+title_suffix
    )
    pl_test = PerplexityMetric(
        corpus=test_corpus, logger="visdom", viz_env=viz_env, title="Perplexity (test)"+title_suffix
    )
    # define other remaining metrics available
    ch_umass = CoherenceMetric(
        corpus=training_corpus, coherence="u_mass", topn=10, logger="visdom", viz_env=viz_env, title="Coherence (u_mass)"+title_suffix
    )
    ch_cv = CoherenceMetric(
        corpus=training_corpus, texts=documents, coherence="c_v", topn=10, logger="visdom", viz_env=viz_env, title="Coherence (c_v)"+title_suffix
    )
    diff_kl = DiffMetric(
        distance="kullback_leibler", logger="visdom", viz_env=viz_env, title="Diff (kullback_leibler)"+title_suffix
    )
    convergence_kl = ConvergenceMetric(
        distance="jaccard", logger="visdom", viz_env=viz_env, title="Convergence (jaccard)"+title_suffix
    )
    return [pl_holdout, pl_test, ch_umass, ch_cv, diff_kl, convergence_kl]

building dictionary
split dataset. size of: train_set=20530, val_set=2566, test_set=2567,


In [6]:
def get_parameterset(corpus, dictionary, callbacks=None, nbtopics=100, parametrization='a42', eval_every=None):
    print(f'building LDA model "{parametrization}" with {nbtopics} number of topics')
    default = dict(random_state=42, corpus=corpus, id2word=dictionary, num_topics=nbtopics, 
                   eval_every=eval_every, callbacks=callbacks, chunksize=20_000)
    ldamodels = {
        'a42': dict(passes=10),
        'b42': dict(passes=10, iterations=200),
        'c42': dict(passes=10, iterations=1_000),
        'd42': dict(passes=10, iterations=200, alpha=0.1, eta=0.01),
        'e42': dict(passes=20, iterations=200, alpha='auto', eta='auto'),
    }
    for key, dic in ldamodels.items():
        dic.update(default)
    return ldamodels[parametrization]

Install Visdom via ```pip install visdom``` and run ```python -m visdom.server``` to start the server.
You will be able to view online log stats @ http://localhost:8097/

Attention: gensim is currently not fully compatible with visdom. See https://github.com/RaRe-Technologies/gensim/issues/2155 for details.

To fix this issue change in gensim.models.callbacks.Callback.on_epoch_end the following line
```
self.viz.updateTrace(
    Y=np.array([value]), X=np.array([epoch]), env=metric.viz_env, win=self.windows[i]
)
```
to
```
self.viz.line(
    Y=np.array([value]), X=np.array([epoch]), env=metric.viz_env, win=self.windows[i], update='append'
)
```
You might need to re-import gensim (and probably even restart this notebook).

In [7]:
save = False

In [ ]:
params_list = ['a42', 'b42', 'c42', 'd42', 'e42']
implementations = [
    ('LDAmodel', LdaModel),
    ('LDAmulticore', LdaMulticore),
    ('LDAmallet', LdaMallet),
]
choice = 2
model_name = implementations[choice][0] + '--test'
UsedModel = implementations[choice][1]
metrics = []
topn = 20
# params = params_list[3]
env_id = f"{dataset}-{model_name}"
for params in params_list[:1]:
    for nbtopics in [10, 25, 50, 100][:1]:  # range(10, 101, 10):
        # Choose α from [0.05, 0.1, 0.5, 1, 5, 10]
        # Choose β from [0.05, 0.1, 0.5, 1, 5, 10]
        callbacks = init_callbacks(
            viz_env=env_id,
            title_suffix=f", {params}, {nbtopics}tpx"
        )
        kwargs = get_parameterset(
            training_corpus,
            dictionary,
            callbacks=callbacks,
            nbtopics=nbtopics,
            parametrization=params
        )
        if 'multicore' in model_name:
            kwargs['workers'] = 3
            kwargs.pop('callbacks', None)
        elif 'mallet' in model_name:
            kwargs['workers'] = 3
            kwargs['mallet_path'] = join(DATA_BASE, 'Mallet', 'bin', 'mallet')
            kwargs.pop('passes', None)
            kwargs.pop('random_state', None)
            kwargs.pop('eval_every', None)
            kwargs.pop('callbacks', None)
            kwargs.pop('chunksize', None)

        print('running {model_name}')
        ldamodel = UsedModel(**kwargs)

        topics = [
            [dataset] +
            [dictionary[term[0]] for term in ldamodel.get_topic_terms(i, topn=topn)]
            for i in range(nbtopics)
        ]
        df_lda = pd.DataFrame(topics, columns=['dataset'] + ['term' + str(i) for i in range(topn)])

        current_metrics = ldamodel.metrics
        # print(current_metrics)
        metrics.append(('env_id', current_metrics))

        if save:
            out_dir = join(ETL_PATH, f'{model_name}/{params}')
            if not exists(out_dir):
                makedirs(out_dir)
            out = join(out_dir, f'{dataset}_{model_name}_{params}_{nbtopics}')
            print('saving to', out)
            df_lda.to_csv(out + '.csv')
            ldamodel.save(out)
            with open(out + '_stats.json', 'w') as fp:
                json.dump(stats, fp)
            with open(out + '_metrics.json', 'w') as fp:
                serializable_metrics = {}
                for k, v in current_metrics.items():
                    if isinstance(v[0], np.ndarray):
                        serializable_metrics[k] = [x.tolist() for x in v]
                    else:
                        serializable_metrics[k] = [float(x) for x in v]
                json.dump(serializable_metrics, fp)
            vis.save([env_id])


-----

### Evaluation

In [18]:
import pyLDAvis as ldavis
import pyLDAvis.gensim

ldavis.enable_notebook()
prepared_data = ldavis.gensim.prepare(ldamodel, training_corpus, dictionary)
prepared_data

PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
30     8.078730        1       1 -0.326306 -0.206703
43     5.374687        1       2  0.185581 -0.233256
42     4.590188        1       3 -0.011230  0.049386
22     3.893477        1       4 -0.272089 -0.185167
16     3.133803        1       5  0.177576 -0.162390
6      3.120531        1       6  0.067090 -0.045838
39     2.595543        1       7  0.150075 -0.119501
10     2.354005        1       8  0.097501 -0.019771
33     2.335376        1       9 -0.234601 -0.112315
2      2.175225        1      10  0.031185 -0.046346
44     1.971773        1      11  0.109320 -0.043679
41     1.932203        1      12 -0.063810 -0.036019
1      1.877694        1      13 -0.151878 -0.060330
38     1.773790        1      14  0.076567 -0.011897
49     1.772362        1      15  0.089651 -0.049244
31     1.767273        1      16  0.029875  0.014077
11     1.765947        1      17 -0.066477  0.013248
12     1.765345        1      18 -0.092848  0.001502
18     1.754438        1      19  0.054654 -0.008624
29     1.749918        1      20  0.102986  0.004911
15     1.740344        1      21  0.082737 -0.045577
27     1.738011        1      22  0.103926 -0.047358
0      1.688985        1      23  0.109195 -0.060440
14     1.664957        1      24  0.085150  0.011540
17     1.643342        1      25  0.063026 -0.038588
28     1.630311        1      26 -0.067697  0.083742
5      1.629337        1      27  0.047733  0.030249
13     1.619340        1      28 -0.058929  0.086414
24     1.608458        1      29 -0.077566  0.053889
23     1.590880        1      30  0.053558 -0.023922
21     1.585594        1      31  0.009159  0.121847
26     1.554661        1      32 -0.086175  0.045405
37     1.541534        1      33 -0.091014 -0.015463
19     1.531816        1      34  0.098605  0.025750
47     1.480445        1      35 -0.006969  0.071471
4      1.478785        1      36  0.062056  0.029485
25     1.476843        1      37 -0.044389  0.111183
20     1.456000        1      38 -0.062007 -0.028600
46     1.451211        1      39 -0.090030  0.031236
35     1.429834        1      40 -0.023543  0.065292
7      1.414581        1      41 -0.030326  0.097061
40     1.371469        1      42  0.042454  0.047562
45     1.367851        1      43 -0.012799  0.060313
48     1.350683        1      44  0.047677  0.018837
8      1.247318        1      45  0.058374  0.040000
32     1.236718        1      46 -0.075727  0.077556
36     1.225097        1      47 -0.020310  0.104717
3      1.187473        1      48 -0.045216  0.071976
34     1.164683        1      49  0.015531  0.136558
9      1.111130        1      50 -0.039302  0.095823, topic_info=      Category         Freq                 Term        Total  loglift  \
term                                                                     
314    Default  4006.000000               Radweg  4006.000000  30.0000   
201    Default  2282.000000                 Kind  2282.000000  29.0000   
78     Default  3287.000000                   Re  3287.000000  28.0000   
445    Default  3804.000000            Radfahrer  3804.000000  27.0000   
5      Default  2424.000000              Beitrag  2424.000000  26.0000   
6      Default  1940.000000                 Dank  1940.000000  25.0000   
20     Default  1462.000000            Kategorie  1462.000000  24.0000   
185    Default  1163.000000                  Bad  1163.000000  23.0000   
13     Default  3985.000000            Vorschlag  3985.000000  22.0000   
4      Default  4865.000000                Stadt  4865.000000  21.0000   
76     Default  1410.000000                Platz  1410.000000  20.0000   
295    Default  1478.000000                  Weg  1478.000000  19.0000   
9      Default  1344.000000           Moderation  1344.000000  18.0000   
70     Default  2803.000000                 Köln  2803.000000  17.0000   
12     Default  1